In [ ]:
import os
import re
import glob
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import rasterio
from rasterio.transform import from_origin
import os


# 降雨数据转栅格

In [ ]:

# ----------------- USER PARAMETERS -----------------
FOLDER = r'data/grid1km_gd_rain_xls_2024'  # updated folder path
TARGET_STR = '2024061523'  # target timestamp in filenames YYYYMMDDHH
output_path = r"../out/rain_cumulative_2024061523.csv"
# ---------------------------------------------------

WINDOWS = {
    'rain_cum_3d': 72,
    'rain_cum_2d': 48,
    'rain_cum_1d': 24,
    'rain_cum_2h': 2,
    'rain_cum_3h': 3,
    'rain_cum_4h': 4,
    'rain_cum_5h': 5,
    'rain_cum_6h': 6,
    'rain_cum_7h': 7,
}

def parse_yyyymmddhh(s: str) -> datetime:
    year = int(s[0:4]); month = int(s[4:6]); day = int(s[6:8]); hour = int(s[8:10])
    if hour == 24:
        dt = datetime(year, month, day) + timedelta(days=1)
    else:
        dt = datetime(year, month, day, hour)
    return dt

pattern = os.path.join(FOLDER, '*.xlsx')
files = glob.glob(pattern)
if not files:
    raise SystemExit(f'No .xlsx files found in folder: {FOLDER}')

file_dt_map = {}
filename_re = re.compile(r'(\d{10})')
for fp in files:
    bn = os.path.basename(fp)
    m = filename_re.search(bn)
    if not m:
        continue
    ts_str = m.group(1)
    try:
        dt = parse_yyyymmddhh(ts_str)
    except Exception as e:
        print(f"Warning: couldn't parse timestamp from {bn}: {e}")
        continue
    file_dt_map[dt] = fp

target_dt = parse_yyyymmddhh(TARGET_STR)
if target_dt not in file_dt_map:
    raise SystemExit(f"Target file for {TARGET_STR} not found in folder.")

target_fp = file_dt_map[target_dt]
print('Loading target file:', target_fp)
df_target = pd.read_excel(target_fp, engine='openpyxl')
required_cols = ['URI', 'lon', 'lat', 'province', 'city', 'Rain']
missing_cols = [c for c in required_cols if c not in df_target.columns]
if missing_cols:
    raise SystemExit(f'Target file missing columns: {missing_cols}')

base = df_target[['URI','lon','lat','province']].drop_duplicates(subset=['URI']).set_index('URI')
base = base.sort_index()

series_by_dt = {}
needed_dts = set()
for hours in WINDOWS.values():
    for h in range(1, hours+1):
        needed_dts.add(target_dt - timedelta(hours=h))
needed_dts = sorted(needed_dts)

print(f'Total unique hourly files needed: {len(needed_dts)}')

for dt in needed_dts:
    if dt in file_dt_map:
        fp = file_dt_map[dt]
        try:
            dfi = pd.read_excel(fp, engine='openpyxl')
        except Exception as e:
            print(f"Warning: failed to read {fp}: {e}.")
            series_by_dt[dt] = pd.Series(index=base.index, data=float('nan'))
            continue
        if 'URI' not in dfi.columns or 'Rain' not in dfi.columns:
            print(f"Warning: file {fp} missing URI or Rain columns.")
            series_by_dt[dt] = pd.Series(index=base.index, data=float('nan'))
            continue
        s = dfi.set_index('URI')['Rain'].reindex(base.index)
        series_by_dt[dt] = s
    else:
        print(f"Warning: hourly file for {dt} not found.")
        series_by_dt[dt] = pd.Series(index=base.index, data=float('nan'))

out = base.copy()
for colname, hours in WINDOWS.items():
    dts = [target_dt - timedelta(hours=h) for h in range(1, hours+1)]
    frames = [series_by_dt[d] for d in dts]
    if frames:
        df_concat = pd.concat(frames, axis=1)
        out[colname] = df_concat.sum(axis=1, skipna=True, min_count=1)
    else:
        out[colname] = float('nan')

out_reset = out.reset_index()
keep_cols = ['URI','lon','lat','province'] + list(WINDOWS.keys())
out_final = out_reset[keep_cols]
df = out_final
base_cols = ['URI', 'lon', 'lat', 'province']
rain_cols = [col for col in df.columns if col not in base_cols]
df_no_neg = df[(df[rain_cols] >= 0).all(axis=1)]
df_no_neg.to_csv(output_path, index=False, encoding='utf-8-sig')
print('Done.')

Loading target file: data/grid1km_gd_rain_xls_2024\grid1km_obs_rain_2024061523.xlsx
Total unique hourly files needed: 72


In [ ]:
# 输入 CSV
csv_path = r"../out/rain_cumulative_2024061523.csv"

# 输出文件夹
out_dir = r"../out/grid1km_gd_rain_xls_2024/rain_rasters"
os.makedirs(out_dir, exist_ok=True)

# 读取数据
df = pd.read_csv(csv_path)

# 基础列
base_cols = ['URI', 'lon', 'lat', 'province']

# 累计降雨量列
rain_cols = [col for col in df.columns if col not in base_cols]

# 获取唯一经纬度并排序
lons = np.sort(df['lon'].unique())
lats = np.sort(df['lat'].unique())[::-1]  # 从北到南

# 分辨率（假设经纬度为度）
res_x = np.abs(lons[1] - lons[0])
res_y = np.abs(lats[1] - lats[0])

# 创建 transform
transform = from_origin(west=lons.min() - res_x/2,
                        north=lats.max() + res_y/2,
                        xsize=res_x,
                        ysize=res_y)

# 行列数
height = len(lats)
width = len(lons)

# 对每个累计降雨量列生成栅格
for col in rain_cols:
    # 初始化空阵
    arr = np.full((height, width), np.nan, dtype=np.float32)

    # 填值
    for _, row in df.iterrows():
        x_idx = np.where(lons == row['lon'])[0][0]
        y_idx = np.where(lats == row['lat'])[0][0]
        arr[y_idx, x_idx] = row[col]

    # 输出路径
    out_path = os.path.join(out_dir, f"{col}_2024061523.tif")

    # 保存 GeoTIFF
    with rasterio.open(
        out_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=arr.dtype,
        crs='EPSG:4326',  # WGS84
        transform=transform,
        nodata=np.nan
    ) as dst:
        dst.write(arr, 1)

    print(f"已生成: {out_path}")
